In [1]:
import pydicom # for reading dicom files
import os # for doing directory operations 
import pandas as pd # for some simple data analysis (right now, just to load in the labels data and quickly reference it)
import cv2
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt


# Change this to wherever you are storing your data:
# IF YOU ARE FOLLOWING aaON KAGGLE, YOU CAN ONLY PLAY WITH THE SAMPLE DATA, WHICH IS MUCH SMALLER
patientsPAths = []
for i in range(15):
    s1 = "3Dircadb1/3Dircadb1."
    s2 = "/PATIENT_DICOM"
    patientsPAths.append(s1+str(i+1)+s2)

In [2]:
slicesData =[]
for patient in patientsPAths:
#     labels_df.get_value(patient, 'cancer')
    path = patient
    # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
    slices = [pydicom.read_file(path+"/" + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    slicesData.append(slices)

In [3]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [4]:
patients_pixels = []
first_patient_pixels = []
for patients in slicesData:
    first_patient = patients
    first_patient_pixels = get_pixels_hu(first_patient)
    patients_pixels.append(first_patient_pixels)
    #plt.hist(first_patient_pixels.flatten(), bins=100, color='c')
    #plt.xlabel("Hounsfield Units (HU)")
    #plt.ylabel("Frequency")
    #plt.show()
  

In [5]:
MIN_BOUND = -200.0
MAX_BOUND = 500.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

normalized_patients = []
for pat in patients_pixels:
    normalized_patients.append( normalize(pat) )


In [6]:
#for patient in normalized_patients:
#     first_patient = patients
#     first_patient_pixels = get_pixels_hu(first_patient)
    #print(patient.shape)
    #plt.hist(patient.flatten(), bins=80, color='c')
    #plt.xlabel("Hounsfield Units (HU)")
    #plt.ylabel("Frequency")
    #plt.show()
    # Show some slice in the middle
    #plt.imshow(patient[len(patient)-50], cmap=plt.cm.gray)
    #plt.show()

In [7]:
liver_segm = []
for i in range(15):
    s1 = "3Dircadb1/3Dircadb1."
    s2 = "/MASKS_DICOM/liver"
    liver_segm.append(s1+str(i+1)+s2)
#print(liver_segm)

In [8]:
liver_slices =[]
for liver in liver_segm:
#     labels_df.get_value(patient, 'cancer')
    path = liver
    # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
    slices = [pydicom.read_file(path+"/" + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    liver_slices.append(slices)

#liver_slices[6][1]

In [9]:
liver_pixels = []
for liver in liver_slices:
    pixels = get_pixels_hu(liver)
    liver_pixels.append(pixels)
    #plt.hist(pixels.flatten(), bins=200, color='c')
    #plt.xlabel("Hounsfield Units (HU)")
    #plt.ylabel("Frequency")
    #plt.show()
    # Show some slice in the middle
    # plt.imshow(first_patient_pixels[len(first_patient)-1], cmap=plt.cm.gray)
    # plt.show()

In [10]:
def normalize2(image):
    MAX_BOUND = np.max(image)
    MIN_BOUND = np.min(image)
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

normalized_liver = []
for pat in liver_pixels:
    normalized_liver.append( normalize2(pat) )
    
#for patient in normalized_liver:
#     first_patient = patients
#     first_patient_pixels = get_pixels_hu(first_patient)
#    print(patient.shape)
#    plt.hist(patient.flatten(), bins=10, color='c')
#    plt.xlabel("Hounsfield Units (HU)")
#    plt.ylabel("Frequency")
#    plt.show()
#    # Show some slice in the middle
#    plt.imshow(patient[len(patient)-27], cmap=plt.cm.gray)
#    plt.show()

In [11]:
IMG_PX_SIZE = 1000

# fig = plt.figure()
#for num,each_slice in enumerate(normalized_liver[14]):
#   y = fig.add_subplot(,6,num+1)
    #print(num)
    #plt.imshow(np.array(each_slice),cmap=plt.cm.gray)
    #plt.show()


In [12]:
def tumor_segm(tumur_count = [7,1,1,1,1,3,1,1,1,1,1,1,2,1,1]):
    tumor_segm = []
    pathes = []
    for i in range(15):
        s1 = "3Dircadb1/3Dircadb1."
        s2 = "/MASKS_DICOM/livertumor0"
        if tumur_count[i] == 1:
            pathes.append([s1+str(i+1)+s2+'1'])
        else:
            l = []
            for k in range(tumur_count[i]):
                l.append(s1+str(i+1)+s2+str(k+1))
            pathes.append(l)
    #print(pathes)
    
    
    tumor_slices = []
    tumor_norm = []
    for path in pathes:
    #     labels_df.get_value(patient, 'cancer')
        if len(path) == 1:
            # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
            slices = [pydicom.read_file(path[0]+"/" + s) for s in os.listdir(path[0])]
            slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
            tumor_slices.append(slices)
            hf_3d = get_pixels_hu(slices)
            norm = normalize2(hf_3d)
            tumor_norm.append(norm)
            
        else:
            temp = []
            slices1 = [pydicom.read_file(path[0]+"/" + s) for s in os.listdir(path[0])]
            slices1.sort(key = lambda x: int(x.ImagePositionPatient[2]))
            dumy = get_pixels_hu(slices1)
            add = np.zeros(shape = dumy.shape)
            for k in range(len(path)):
                slices = [pydicom.read_file(path[k]+"/" + s) for s in os.listdir(path[k])]
                slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
                temp.append(slices)
                hf_3d = get_pixels_hu(slices)
                norm = normalize2(hf_3d)
                add = add + norm
            tumor_slices.append(temp)
            tumor_norm.append(add)
    
    
    return tumor_norm,tumor_slices

normalized_tumor,tumor_slices = tumor_segm()

In [13]:
#for patient in normalized_tumor:
#     first_patient = patients
#     first_patient_pixels = get_pixels_hu(first_patient)
    #print(patient.shape)
    #plt.hist(patient.flatten(), bins=100, color='c')
    #plt.xlabel("Hounsfield Units (HU)")
    #plt.ylabel("Frequency")
    #plt.show()
    # Show some slice in the middle
    #plt.imshow(patient[len(patient)-27], cmap=plt.cm.gray)
    #plt.show()

In [14]:
#np.save("tumorsOfPatient1", fullTumor)

In [19]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    #spacing = np.array( shape=([scan[0].SliceThickness] , scan[0].PixelSpacing, scan[0].PixelSpacing), dtype=np.float32)
    spacing = np.array([slices[0].SliceThickness, slices[0].PixelSpacing[0], slices[0].PixelSpacing[1]], dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [ ]:
resampled_patients_pixels = []
for i in range(15):
    pix_resampled, spacing = resample(normalized_patients[i], slicesData[i], [1,1,1])
    resampled_patients_pixels.append(pix_resampled)
    print('patient ',i)
    print("Shape after resampling\t", pix_resampled.shape)
    #plt.imshow(pix_resampled[int(pix_resampled.shape[0]/2)])
    #plt.show()

patient  0
Shape after resampling	 (516, 360, 360)
patient  1
Shape after resampling	 (688, 360, 360)
patient  2
Shape after resampling	 (800, 360, 360)
patient  3
Shape after resampling	 (364, 360, 360)
patient  4
Shape after resampling	 (540, 360, 360)
patient  5
Shape after resampling	 (496, 360, 360)
patient  6
Shape after resampling	 (444, 360, 360)
patient  7
Shape after resampling	 (488, 360, 360)
patient  8
Shape after resampling	 (1040, 360, 360)
patient  9
Shape after resampling	 (488, 360, 360)
patient  10
Shape after resampling	 (500, 360, 360)
patient  11
Shape after resampling	 (620, 360, 360)
patient  12
Shape after resampling	 (476, 360, 360)
patient  13
Shape after resampling	 (296, 360, 360)


In [ ]:
np.save("offline-patient-resampled-1", resampled_patients_pixels)
np.savetxt("offline-patient-resampled-txt-1", resampled_patients_pixels)

In [ ]:
resampled_liver_pixels = []
for i in range(15):
    pix_resampled, spacing = resample(normalized_liver[i], slicesData[i], [1,1,1])
    resampled_liver_pixels.append(pix_resampled)
    print('patient ',i)
    print("Shape after resampling\t", pix_resampled.shape)
    #plt.imshow(pix_resampled[int(pix_resampled.shape[0]/2)])
    #plt.show()

In [ ]:
resampled_tumor_pixels = []
for i in range(15):
    pix_resampled, spacing = resample(normalized_liver[i], slicesData[i], [1,1,1])
    resampled_tumor_pixels.append(pix_resampled)
    print('patient ',i)
    print("Shape after resampling\t", pix_resampled.shape)
    plt.imshow(pix_resampled[int(pix_resampled.shape[0]/2)])
    plt.show()